In [4]:
import pandas as pd
import warnings

import numpy as np
warnings.filterwarnings("ignore", category=UserWarning)

# Load the dfset
df = pd.read_csv('AEP_hourly.csv')

# Convert 'Datetime' column to datetime format
df['Datetime'] = pd.to_datetime(df['Datetime'])
# Sort the dfframe by datetime (if needed)
df = df.sort_values('Datetime')

# Display the first few rows of the dfset
print(df.head())
data = df.copy()


                Datetime   AEP_MW
2183 2004-10-01 01:00:00  12379.0
2184 2004-10-01 02:00:00  11935.0
2185 2004-10-01 03:00:00  11692.0
2186 2004-10-01 04:00:00  11597.0
2187 2004-10-01 05:00:00  11681.0


In [6]:
from prophet import Prophet
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming 'data' is your DataFrame containing datetime ('Datetime') and target ('AEP_MW') columns
# Rename columns as required by Prophet ('ds' for datetime, 'y' for target variable)
data_2 = data.rename(columns={'Datetime': 'ds', 'AEP_MW': 'y'})

# Initialize Prophet model with potentially yearly, weekly, and daily seasonality
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    seasonality_mode='additive'
)

# Fit the model
prophet_model.fit(data_2)

# Forecast future values (next 1 month)
future = prophet_model.make_future_dataframe(periods=30 * 24, freq='H')  # Forecasting 30 days (1 month) ahead

# Predict values
forecast = prophet_model.predict(future)

# Filter forecast for the next 1 month (last 30 days)
forecast_next_1_month = forecast.tail(30 * 24)  # Extracting last 30 days of forecast (1 month)

# Extract forecasted values (yhat) and actual values (y) for RMSE calculation
forecast_values = forecast_next_1_month['yhat'].values
actual_values = data_2['y'].values[-len(forecast_values):]

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(actual_values, forecast_values))
print(f"Root Mean Squared Error (RMSE): {rmse}")



20:48:31 - cmdstanpy - INFO - Chain [1] start processing
20:49:46 - cmdstanpy - INFO - Chain [1] done processing


Root Mean Squared Error (RMSE): 2071.116224341403
